### TCC 6 – individual - Tatiana Prado Santos Massoco

# Desenvolvimento de Modelo Preditivo de Risco ESG para Investimentos Sustentáveis

Fonte de daddos: https://www.kaggle.com/datasets/tunguz/environment-social-and-governance-data/data

## Objetivo do Projeto  
Desenvolver um modelo preditivo de risco ESG (Environmental, Social, Governance) para investimentos sustentáveis. O modelo visa identificar riscos associados a fatores ESG, auxiliando na tomada de decisões alinhadas à sustentabilidade.  

---

## Ferramentas Necessárias  
- **Linguagem de Programação:** Python.  
- **Bibliotecas:** pandas, numpy, seaborn, matplotlib, scikit-learn.  
- **Ambiente:** Jupyter Notebook, Anaconda.  
- **Controle de Versão:** Git e GitHub.  
- **Deploy:** Flask, Docker.  
- **Monitoramento:** Prometheus e Grafana.  

---

# Modelo Preditivo

## Passo 7: Desenvolvimento do Modelo Preditivo


**Seleção das Variáveis Alvo**

- Identificar variáveis representativas de cada pilar ESG (Ambiental, Social e Governança).  

**Preparação dos Dados**

- Dividir os dados em conjuntos de treino e teste para garantir uma avaliação consistente dos modelos.  

**Escolha e Treinamento de Algoritmos**

Testar diferentes algoritmos de aprendizado supervisionado, como:  
- **Regressão Linear:** Para variáveis contínuas, como emissões de CO₂.  
- **Random Forest ou Gradient Boosting:** Para modelos mais robustos e flexíveis.  
- **Regressão Logística ou Classificação:** Caso existam variáveis categóricas.  

**Ajuste de Hiperparâmetros**

- Usar técnicas como:  
  - **Busca em Grade (Grid Search):** Para testar combinações sistemáticas de hiperparâmetros.  
  - **Busca Aleatória (Random Search):** Para otimização mais rápida e eficiente.  

**Validação**

- Avaliar os modelos com métricas apropriadas:  
  - **Para Regressão:** R², MAE, MSE.  
  - **Para Classificação:** Acurácia, F1, ROC-AUC.  

**Seleção do Melhor Modelo**

- Escolher o modelo com o melhor desempenho para cada variável alvo com base nos resultados das validações.  

**Documentação**

- Registrar no notebook as decisões tomadas, os códigos implementados e os resultados obtidos durante o treinamento e validação.  


In [24]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# Caminho para o arquivo de dados limpo e normalizado
caminho_dados = "../data/ESGData_clean_final.csv"

# Carregar o dataset
dados = pd.read_csv(caminho_dados)

# Exibir as primeiras linhas para verificar o carregamento
print(dados.head())

  Country Name Country Code  \
0   Arab World          ARB   
1   Arab World          ARB   
2   Arab World          ARB   
3   Arab World          ARB   
4   Arab World          ARB   

                                      Indicator Name     Indicator Code  \
0  Adjusted savings: natural resources depletion ...  NY.ADJ.DRES.GN.ZS   
1  Adjusted savings: net forest depletion (% of GNI)  NY.ADJ.DFOR.GN.ZS   
2                 Agricultural land (% of land area)     AG.LND.AGRI.ZS   
3  Agriculture, forestry, and fishing, value adde...     NV.AGR.TOTL.ZS   
4             CO2 emissions (metric tons per capita)     EN.ATM.CO2E.PC   

        1961       1962       1963       1964       1965       1966  ...  \
0  19.156353  19.804722  20.520000  20.950959  21.033578  21.147132  ...   
1  19.156353  19.804722  20.520000  20.950959  21.033578  21.147132  ...   
2  27.835643  27.826564  27.845522  27.847925  27.866972  27.866171  ...   
3  19.156353  19.804722  20.520000  20.950959  21.033578  

### Análise do Dataset Carregado

Esse output mostra as primeiras cinco linhas do dataset carregado e permite confirmar que os dados foram importados corretamente. Abaixo, segue uma análise do que pude observar:

### Estrutura do Dataset

**Colunas principais**

- **Country Name:** Nome do país ou região.  
- **Country Code:** Código ISO de três letras do país ou região.  
- **Indicator Name:** Nome do indicador, que descreve a métrica ou dado (ex: emissões de CO2, área agrícola, etc).  
- **Indicator Code:** Código curto associado ao indicador (ex.: NY.ADJ.DRES.GN.ZS).  

**Anos**

- Cada coluna numérica (ex: 1961, 1962, etc) representa valores do indicador para um determinado ano.  
- Existem também colunas derivadas de transformações logarítmicas (`1970_log`, `1980_log`, etc).  

**Formato do Dataset**

- Contém 68 colunas, incluindo informações descritivas e valores numéricos para diversos anos.  

---

### Observações Relevantes

**Indicadores e transformações**

- Os valores originais dos anos estão listados, junto com transformações logarítmicas aplicadas para normalização de dados (`1970_log`, `1980_log`, etc.).  
- Isso indica que já foi realizado um pré-processamento para melhorar a análise estatística e a modelagem.  

**Consistência dos dados**

- Todas as colunas numéricas parecem preenchidas com valores (não há sinais evidentes de valores ausentes nas primeiras linhas).  

**Formatos numéricos**

- Algumas colunas, como `2019`, têm valores muito altos, o que pode refletir diferentes escalas entre os indicadores.  

---

### Próximos passos com base nesse Dataset

**Validação dos dados**

- Verificar a presença de valores ausentes ou inconsistências no restante do dataset.  
- Confirmar que as transformações (ex.: colunas `_log`) foram aplicadas corretamente.  

**Exploração e Análise Estatística**

- Gerar estatísticas descritivas para entender a dispersão e distribuição dos valores.  
- Visualizar os dados para identificar padrões, tendências e possíveis outliers.  

**Preparação Adicional**

- Certificar que os dados estejam prontos para a modelagem, incluindo normalização, remoção de redundâncias e seleção de indicadores relevantes.  


In [7]:
# Lista de variáveis alvo para análise
variaveis_alvo = [
    "CO2 emissions (metric tons per capita)",  # Pilar Ambiental
    "Life expectancy at birth, total (years)",  # Pilar Social
    "Gini index (World Bank estimate)"         # Pilar Governança
]

# Iterar sobre cada variável alvo
for target_indicator in variaveis_alvo:
    print(f"\nPreparando os dados para a variável alvo: {target_indicator}")
    
    # Filtrar o dataset para o indicador atual
    dados_filtrados = dados[dados["Indicator Name"] == target_indicator]
    
    # Definir as colunas que devem ser removidas
    colunas_removidas = ["Country Name", "Country Code", "Indicator Name", "Indicator Code"]
    
    # Escolher o ano a ser previsto
    target_column = "2010"  # Escolha o ano desejado para previsão
    
    # Separar recursos (X) e variável alvo (y)
    X = dados_filtrados.drop(columns=colunas_removidas + [target_column])
    y = dados_filtrados[target_column]
    
    # Dividir os dados em treino e teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Verificar os shapes das divisões
    print(f"Shape do conjunto de treino para {target_indicator}: {X_treino.shape}")
    print(f"Shape do conjunto de teste para {target_indicator}: {X_teste.shape}")




Preparando os dados para a variável alvo: CO2 emissions (metric tons per capita)
Shape do conjunto de treino para CO2 emissions (metric tons per capita): (166, 63)
Shape do conjunto de teste para CO2 emissions (metric tons per capita): (42, 63)

Preparando os dados para a variável alvo: Life expectancy at birth, total (years)
Shape do conjunto de treino para Life expectancy at birth, total (years): (182, 63)
Shape do conjunto de teste para Life expectancy at birth, total (years): (46, 63)

Preparando os dados para a variável alvo: Gini index (World Bank estimate)
Shape do conjunto de treino para Gini index (World Bank estimate): (3, 63)
Shape do conjunto de teste para Gini index (World Bank estimate): (1, 63)


### Análise dos resultados da preparação dos dados

Os resultados indicam que os conjuntos de dados para cada variável alvo foram preparados com sucesso, mas há algumas observações importantes a serem feitas:

**Observações:**

**Tamanho dos Conjuntos de Dados**  
- Para as variáveis **"CO2 emissions (metric tons per capita)"** e **"Life expectancy at birth, total (years)"**, os conjuntos de treino e teste têm tamanhos razoáveis para treinar e avaliar modelos preditivos.  
- No caso do **"Gini index (World Bank estimate)"**, o conjunto de dados é extremamente pequeno, com apenas 4 exemplos no total.  
  - Isso pode dificultar a modelagem e avaliação, pois há poucos dados disponíveis para aprendizado.  

**Dimensão das Variáveis (63 colunas)**  
- Cada conjunto de treino e teste possui 63 variáveis (anos), o que oferece uma boa quantidade de informações para o modelo.  

**Discrepância no Número de Dados**  
- A baixa quantidade de dados para o indicador de governança pode estar relacionada à disponibilidade limitada no dataset original.  


In [8]:
# Importar bibliotecas necessárias
from sklearn.model_selection import train_test_split

# Lista de indicadores por pilar ESG
indicadores_esg = [
    # Pilar Ambiental
    "CO2 emissions (metric tons per capita)",
    "Methane emissions (metric tons of CO2 equivalent per capita)",
    "Nitrous oxide emissions (metric tons of CO2 equivalent per capita)",
    "Energy use (kg of oil equivalent per capita)",
    "Fossil fuel energy consumption (% of total)",
    "Electricity production from coal sources (% of total)",
    "Forest area (% of land area)",
    "Adjusted savings: natural resources depletion (% of GNI)",
    
    # Pilar Social
    "Life expectancy at birth, total (years)",
    "Literacy rate, adult total (% of people ages 15 and above)",
    "School enrollment, primary and secondary (gross), gender parity index (GPI)",
    "Prevalence of overweight (% of adults)",
    "Unemployment, total (% of total labor force)",
    "Individuals using the Internet (% of population)",
    "Access to electricity (% of population)",
    
    # Pilar Governança
    "Gini index (World Bank estimate)",
    "Government expenditure on education, total (% of government expenditure)",
    "Ratio of female to male labor force participation rate (%)",
    "Patent applications, residents",
    "Income share held by lowest 20%",
]

# Iterar sobre os indicadores ESG
for target_column in indicadores_esg:
    if target_column in dados["Indicator Name"].values:
        print(f"\nPreparando os dados para a variável alvo: {target_column}")
        
        # Filtrar o dataset apenas para o indicador atual
        dados_filtrados = dados[dados["Indicator Name"] == target_column]
        
        # Separar recursos (X) e variável alvo (y)
        X = dados_filtrados.drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"])
        y = dados_filtrados.iloc[:, 4:]  # Colunas de anos como alvo
        
        # Verificar se há dados suficientes para dividir
        if len(X) > 1:
            # Dividir os dados em treino e teste
            X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
            
            # Exibir os shapes das divisões
            print(f"Shape do conjunto de treino para {target_column}: {X_treino.shape}")
            print(f"Shape do conjunto de teste para {target_column}: {X_teste.shape}")
        else:
            print(f"A variável {target_column} não possui dados suficientes para dividir.")


Preparando os dados para a variável alvo: CO2 emissions (metric tons per capita)
Shape do conjunto de treino para CO2 emissions (metric tons per capita): (166, 64)
Shape do conjunto de teste para CO2 emissions (metric tons per capita): (42, 64)

Preparando os dados para a variável alvo: Methane emissions (metric tons of CO2 equivalent per capita)
Shape do conjunto de treino para Methane emissions (metric tons of CO2 equivalent per capita): (185, 64)
Shape do conjunto de teste para Methane emissions (metric tons of CO2 equivalent per capita): (47, 64)

Preparando os dados para a variável alvo: Nitrous oxide emissions (metric tons of CO2 equivalent per capita)
Shape do conjunto de treino para Nitrous oxide emissions (metric tons of CO2 equivalent per capita): (186, 64)
Shape do conjunto de teste para Nitrous oxide emissions (metric tons of CO2 equivalent per capita): (47, 64)

Preparando os dados para a variável alvo: Energy use (kg of oil equivalent per capita)
Shape do conjunto de tre

### Análise dos resultados da divisão dos dados

Os resultados indicam que, para a maioria dos indicadores, foi possivel dividir os dados em conjuntos de treino e teste com uma quantidade razoável de exemplos. Contudo, há alguns indicadores que apresentam dados insuficientes, como:

**Indicadores com Dados Insuficientes**

- **Gini index (World Bank estimate):** Apenas 4 exemplos no total.  
- **Income share held by lowest 20%:** Apenas 4 exemplos no total.  
- **Government expenditure on education, total (% of government expenditure):** Apenas 14 exemplos no total.  

---

### Análise geral

**Indicadores com Dados Suficientes**

- A maioria dos indicadores possui dados suficientes para modelagem (treino e teste).  
  - Isso é essencial para garantir que o modelo aprenda padrões de forma eficaz.  
- Exemplos como **CO2 emissions (metric tons per capita)** e **Life expectancy at birth, total (years)** têm mais de 200 exemplos no total, garantindo uma boa base para modelagem.  

**Indicadores com Dados Escassos**

- Alguns indicadores têm menos de 20 exemplos no total, como **Income share held by lowest 20%** e **Gini index (World Bank estimate)**.
  - Esses indicadores podem ser problemáticos para modelagem devido ao risco de overfitting e à falta de representatividade.  

**Distribuição de Treino e Teste**

- O código foi bem-sucedido ao dividir os dados em 80% para treino e 20% para teste para a maioria dos casos.  

---

### Próximos Passos

**Focar nos Indicadores com Dados Suficientes**

- Priorizar indicadores como **CO2 emissions**, **Methane emissions**, **Life expectancy**, entre outros com amostras robustas, para as próximas etapas de modelagem.  

**Abordar Indicadores com Dados Escassos**

- Considerar exclusão desses indicadores da modelagem ou buscar fontes adicionais de dados.  
- Alternativamente, explorar métodos específicos, como:
  - **Modelos simplificados:** Regressão linear simples.  
  - **Técnicas de oversampling:** Como SMOTE, para tentar aumentar a representatividade.  

**Iniciar a Modelagem Preditiva**

- Configurar modelos de aprendizado de máquina para os indicadores escolhidos.  
- Comparar métricas de desempenho, como:
  - **Para Regressão:** MAE, RMSE, R².  
  - **Para Classificação:** Accuracy, Precision, Recall.  


## Introdução ao Desenvolvimento do Modelo Preditivo

### Escolha dos algoritmos

Para o desenvolvimento dos modelos preditivos, decidi utilizar diferentes algoritmos que oferecem um equilíbrio entre desempenho, tempo de execução e uso de memória. A escolha de cada algoritmo foi baseada nas características dos dados e na complexidade de cada variável-alvo. Seguem as opções selecionadas:

**Regressão Linear (Simples, Ridge, Lasso):**  
- **Vantagens:** Simples de implementar, rápido de treinar e com baixa utilização de memória.  
- **Desvantagens:** Desempenho limitado em dados não lineares ou com muitos outliers.  
- **Aplicação:** Ideal para variáveis com comportamento linear, como *Energy use* ou *CO2 emissions*.  

**Árvores de Decisão:**  
- **Vantagens:** Fácil de interpretar e adaptável a dados não lineares.  
- **Desvantagens:** Propensa ao overfitting em datasets pequenos e pode exigir mais memória com árvores profundas.  
- **Aplicação:** Adequada para variáveis com padrões não lineares ou categóricas.  

**Random Forest:**  
- **Vantagens:** Robusto, resistente a overfitting e adaptável a dados não lineares.  
- **Desvantagens:** Tempo de treino mais elevado e maior uso de memória, especialmente com muitos estimadores.  
- **Aplicação:** Recomendado para variáveis complexas, como *Forest area* ou *Fossil fuel consumption*.  

**Gradient Boosting (XGBoost):**  
- **Vantagens:** Excelente desempenho em dados não lineares e controles eficientes para evitar overfitting.  
- **Desvantagens:** Elevado tempo de execução e uso de memória.  
- **Aplicação:** Indicados para variáveis com padrões complexos e datasets moderados, como o *Gini index*.  

---

### Plano para os testes

Para garantir um processo sistemático e eficiente, os modelos serão treinados e avaliados em todas as variáveis-alvo de forma iterativa, seguindo o plano abaixo:

**Definir os Modelos:**  
- Iniciar com Regressão Linear e Árvores de Decisão para testar variáveis com padrões mais claros.  
- Adicionar Random Forest e XGBoost para variáveis com maior complexidade.  

**Treinar e Avaliar:**  
- Dividir os dados em treino e teste para cada variável-alvo.  
- Avaliar o desempenho utilizando métricas como:  
  - **R² (coeficiente de determinação):** Mede a qualidade do ajuste do modelo.  
  - **RMSE (erro médio quadrático):** Avalia a magnitude média dos erros.  
  - **MAE (erro absoluto médio):** Mede a precisão do modelo.  

**Processo de Execução:**  
- Separar as variáveis-alvo, removendo colunas irrelevantes.  
- Normalizar os dados para modelos que exigem escalas semelhantes.  
- Treinar cada modelo para todas as variáveis-alvo.  
- Consolidar e documentar os resultados.  

---

### O que o código executa

**Preparação dos Dados:**  
- Filtra os dados com base na coluna `Indicator Name` para selecionar apenas as observações relacionadas à variável-alvo.  
- Remove colunas não relacionadas ao objetivo da análise, como`Country Name`, `Country Code`, `Indicator Name` e `Indicator Code`.  
- Para variáveis-alvo temporais, calcula a média dos valores por país/ano, reduzindo redundâncias e garantindo que os dados estejam prontos para os algoritmos.  

**Aplicação dos Modelos:**  
- Testa diferentes algoritmos de machine learning, como:  
  - **Regressão Linear**  
  - **Árvores de Decisão**  
  - **Random Forest**  
  - **XGBoost**  
- Os algoritmos foram escolhidos com base nas características das variáveis-alvo e no equilíbrio entre tempo de execução, memória utilizada e precisão esperada.  

**Avaliação do Desempenho:**  
- Calcula métricas importantes para avaliar a eficiência e a precisão dos modelos:  
  - **R² (Coeficiente de Determinação):** Mede a proporção de variação explicada pelo modelo.  
  - **RMSE (Raiz do Erro Quadrático Médio):** Indica o erro médio em termos das unidades da variável-alvo.  
  - **MAE (Erro Médio Absoluto):** Avalia a diferença média entre valores previstos e reais.  

**Resultados Consolidados:**  
- Organiza e exibe as métricas de desempenho para cada modelo e variável-alvo.  
- Facilita a identificação de algoritmos mais adequados para diferentes tipos de dados.  

**Objetivo do Planejamento Estruturado:**  
- Garantir um processo reprodutível e bem documentado.  
- Identificar os algoritmos que melhor se adaptam a cada variável-alvo, otimizando a performance geral do modelo preditivo e respeitando os recursos computacionais disponíveis.  


In [12]:
# Lista das variáveis-alvo
variaveis_alvo = [
    "CO2 emissions (metric tons per capita)",  
    "Methane emissions (metric tons of CO2 equivalent per capita)",  
    "Nitrous oxide emissions (metric tons of CO2 equivalent per capita)",  
    "Energy use (kg of oil equivalent per capita)",  
    "Fossil fuel energy consumption (% of total)",  
    "Electricity production from coal sources (% of total)",  
    "Forest area (% of land area)",  
    "Adjusted savings: natural resources depletion (% of GNI)",  
    "Life expectancy at birth, total (years)",  
    "Literacy rate, adult total (% of people ages 15 and above)",  
    "School enrollment, primary and secondary (gross), gender parity index (GPI)",  
    "Prevalence of overweight (% of adults)",  
    "Individuals using the Internet (% of population)",  
    "Access to electricity (% of population)",  
    "Gini index (World Bank estimate)",  
    "Government expenditure on education, total (% of government expenditure)",  
    "Patent applications, residents",  
    "Income share held by lowest 20%"  
]

# Iterar sobre as variáveis-alvo para preparar e treinar os modelos
for target_column in variaveis_alvo:
    print(f"\nTreinando para variável-alvo: {target_column}")
    
    # Filtrar os dados para a variável-alvo específica
    filtro_alvo = dados["Indicator Name"] == target_column
    dados_alvo = dados[filtro_alvo]
    
    # Preparar os dados (X) e calcular a média dos anos para a variável alvo (y)
    X = dados_alvo.iloc[:, 4:].mean(axis=1).values.reshape(-1, 1)  # Considera as colunas de anos
    y = dados_alvo.iloc[:, 4:].mean(axis=1)  # Calcula a média da variável-alvo

    # Dividir os dados em treino e teste
    try:
        X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
        print(f"Shape do conjunto de treino para {target_column}: {X_treino.shape}")
        print(f"Shape do conjunto de teste para {target_column}: {X_teste.shape}")
    except ValueError as e:
        print(f"Erro ao dividir os dados para {target_column}: {e}")
        continue



Treinando para variável-alvo: CO2 emissions (metric tons per capita)
Shape do conjunto de treino para CO2 emissions (metric tons per capita): (166, 1)
Shape do conjunto de teste para CO2 emissions (metric tons per capita): (42, 1)

Treinando para variável-alvo: Methane emissions (metric tons of CO2 equivalent per capita)
Shape do conjunto de treino para Methane emissions (metric tons of CO2 equivalent per capita): (185, 1)
Shape do conjunto de teste para Methane emissions (metric tons of CO2 equivalent per capita): (47, 1)

Treinando para variável-alvo: Nitrous oxide emissions (metric tons of CO2 equivalent per capita)
Shape do conjunto de treino para Nitrous oxide emissions (metric tons of CO2 equivalent per capita): (186, 1)
Shape do conjunto de teste para Nitrous oxide emissions (metric tons of CO2 equivalent per capita): (47, 1)

Treinando para variável-alvo: Energy use (kg of oil equivalent per capita)
Shape do conjunto de treino para Energy use (kg of oil equivalent per capita):

### Análise dos Resultados da Preparação de Dados

Os resultados acima mostram o processo de preparação e separação dos conjuntos de treino e teste para cada variável-alvo no projeto. Aqui estão algumas observações e os próximos passos:

**Distribuição dos Dados:**  
- Para algumas variáveis, como **"Gini index"** e **"Income share held by lowest 20%"**, há uma quantidade muito limitada de dados, o que pode impactar negativamente a qualidade e confiabilidade dos modelos preditivos.  
- Variáveis como **"CO2 emissions"** e **"Methane emissions"** possuem tamanhos de conjunto mais robustos, o que deve resultar em melhores estimativas e avaliações.  

**Shape dos Conjuntos:**  
- Cada variável-alvo é tratada individualmente, com os dados organizados em uma única coluna (shape: `(n, 1)`).  
- A divisão em treino e teste parece estar consistente com a proporção definida (80% treino e 20% teste).  

**Variáveis com Poucos Dados:**  
- Para variáveis com apenas 3 ou 4 observações, como **"Gini index"**, pode ser necessário considerar estratégias alternativas:  
  - Complementar os dados com fontes adicionais.  
  - Utilizar modelos mais simples ou técnicas estatísticas específicas para lidar com dados escassos.  
  - Excluir essas variáveis do modelo final, dependendo da relevância e desempenho durante os testes.  

---

### Próximos Passos

**Treinamento Inicial dos Modelos:**  
- Testar os modelos básicos (**Regressão Linear**, **Árvores de Decisão**) para cada variável-alvo com os dados disponíveis.  
- Avaliar as métricas (**R²**, **RMSE**, **MAE**) para determinar a adequação dos dados e algoritmos.  

**Análise de Desempenho:**  
- Identificar variáveis com baixa performance nos modelos iniciais.  
- Analisar padrões nos resultados para ajustar estratégias, como a aplicação de **Random Forest** ou **XGBoost** para variáveis mais complexas.  

**Revisão de Dados com Pouca Representatividade:**  
- Definir como tratar variáveis com poucos dados antes de avançar para modelos mais avançados.  
- Decidir entre manter, ajustar ou excluir essas variáveis para otimizar os recursos e resultados.  


In [18]:
# Lista de variáveis-alvo
variaveis_alvo = [
    "CO2 emissions (metric tons per capita)",
    "Methane emissions (metric tons of CO2 equivalent per capita)",
    "Nitrous oxide emissions (metric tons of CO2 equivalent per capita)",
    "Energy use (kg of oil equivalent per capita)",
    "Fossil fuel energy consumption (% of total)",
    "Electricity production from coal sources (% of total)",
    "Forest area (% of land area)",
    "Adjusted savings: natural resources depletion (% of GNI)",
    "Life expectancy at birth, total (years)",
    "Literacy rate, adult total (% of people ages 15 and above)",
    "School enrollment, primary and secondary (gross), gender parity index (GPI)",
    "Prevalence of overweight (% of adults)",
    "Individuals using the Internet (% of population)",
    "Access to electricity (% of population)",
    "Gini index (World Bank estimate)",
    "Government expenditure on education, total (% of government expenditure)",
    "Patent applications, residents",
    "Income share held by lowest 20%"
]

# DataFrame para consolidar os resultados
resultados = []

# Treinar modelos para cada variável-alvo
for target_column in variaveis_alvo:
    print(f"\nTreinando modelos para a variável-alvo: {target_column}")
    
    # Filtrar dados apenas para a variável-alvo
    dados_filtrados = dados[dados["Indicator Name"] == target_column]
    
    # Preparar os dados
    X = dados_filtrados.iloc[:, 4:]  # Anos como recursos
    y = dados_filtrados.iloc[:, 4:].mean(axis=1)  # Média por país/ano
    
    # Garantir que X e y têm o mesmo índice
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    # Dividir os dados em treino e teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Exibir shapes dos conjuntos de treino e teste
    print(f"Shape do conjunto de treino para {target_column}: {X_treino.shape}")
    print(f"Shape do conjunto de teste para {target_column}: {X_teste.shape}")
    
    # Modelos a serem testados
    modelos = {
        "Linear Regression": LinearRegression(),
        "Decision Tree": DecisionTreeRegressor(random_state=42),
        "Random Forest": RandomForestRegressor(random_state=42, n_estimators=100),
        "XGBoost": XGBRegressor(random_state=42, verbosity=0)
    }
    
    for nome_modelo, modelo in modelos.items():
        try:
            # Treinar o modelo
            modelo.fit(X_treino, y_treino)
            # Fazer previsões
            previsoes = modelo.predict(X_teste)
            
            # Calcular métricas de desempenho
            r2 = r2_score(y_teste, previsoes)
            rmse = np.sqrt(mean_squared_error(y_teste, previsoes))
            mae = mean_absolute_error(y_teste, previsoes)
            
            # Adicionar resultados ao DataFrame
            resultados.append({
                "Variável Alvo": target_column,
                "Modelo": nome_modelo,
                "R²": r2,
                "RMSE": rmse,
                "MAE": mae
            })
            print(f"{nome_modelo} - R²: {r2:.3f}, RMSE: {rmse:.3f}, MAE: {mae:.3f}")
        except Exception as e:
            print(f"Erro ao treinar o modelo {nome_modelo} para {target_column}: {e}")

# Converter resultados para um DataFrame e exibir
df_resultados = pd.DataFrame(resultados)
print("\nResultados consolidados:")
print(df_resultados)


Treinando modelos para a variável-alvo: CO2 emissions (metric tons per capita)
Shape do conjunto de treino para CO2 emissions (metric tons per capita): (166, 64)
Shape do conjunto de teste para CO2 emissions (metric tons per capita): (42, 64)
Linear Regression - R²: 1.000, RMSE: 0.000, MAE: 0.000
Decision Tree - R²: 0.888, RMSE: 1.749, MAE: 0.938
Random Forest - R²: 0.956, RMSE: 1.096, MAE: 0.553
XGBoost - R²: 0.924, RMSE: 1.436, MAE: 0.734

Treinando modelos para a variável-alvo: Methane emissions (metric tons of CO2 equivalent per capita)
Shape do conjunto de treino para Methane emissions (metric tons of CO2 equivalent per capita): (185, 64)
Shape do conjunto de teste para Methane emissions (metric tons of CO2 equivalent per capita): (47, 64)
Linear Regression - R²: 1.000, RMSE: 0.000, MAE: 0.000
Decision Tree - R²: 0.901, RMSE: 0.235, MAE: 0.094
Random Forest - R²: 0.975, RMSE: 0.118, MAE: 0.052
XGBoost - R²: 0.938, RMSE: 0.186, MAE: 0.074

Treinando modelos para a variável-alvo: N

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Random Forest - R²: nan, RMSE: 5.497, MAE: 5.497
XGBoost - R²: nan, RMSE: 6.351, MAE: 6.351

Treinando modelos para a variável-alvo: Government expenditure on education, total (% of government expenditure)
Shape do conjunto de treino para Government expenditure on education, total (% of government expenditure): (11, 64)
Shape do conjunto de teste para Government expenditure on education, total (% of government expenditure): (3, 64)
Linear Regression - R²: 0.992, RMSE: 0.074, MAE: 0.056
Decision Tree - R²: -1.912, RMSE: 1.401, MAE: 1.217
Random Forest - R²: 0.995, RMSE: 0.059, MAE: 0.051
XGBoost - R²: 0.741, RMSE: 0.418, MAE: 0.414

Treinando modelos para a variável-alvo: Patent applications, residents
Shape do conjunto de treino para Patent applications, residents: (77, 64)
Shape do conjunto de teste para Patent applications, residents: (20, 64)
Linear Regression - R²: 1.000, RMSE: 0.000, MAE: 0.000
Decision Tree - R²: 1.000, RMSE: 903.710, MAE: 545.618
Random Forest - R²: 0.998, RMSE:

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


## Análise dos Resultados Consolidados

Os resultados consolidados mostram um desempenho variado entre os modelos para cada variável-alvo, o que reflete as características específicas de cada conjunto de dados:

---

### Observações Importantes

**Desempenho da Regressão Linear:**  
- **R²** igual a 1 para quase todas as variáveis, indicando um possível problema de overfitting ou baixa variabilidade nos dados.  
- Precisa ser investigado, especialmente para conjuntos de dados pequenos.  

**Árvores de Decisão e Random Forest:**  
- Desempenho robusto em muitas variáveis, mas com variação significativa nas métricas dependendo da complexidade dos dados.  
- Melhor equilíbrio entre **R²**, **RMSE** e **MAE** foi observado com **Random Forest** para variáveis como *Forest area* e *Life expectancy*.  

**XGBoost:**  
- Excelente desempenho para várias variáveis, mas piora em algumas como *Nitrous oxide emissions* e *Patent applications*.  
- Sugere que a escolha de hiperparâmetros pode ser crítica.  

**Variáveis com Dados Limitados:**  
- Variáveis como *Gini index* e *Income share held by lowest 20%* apresentam métricas não confiáveis devido ao tamanho reduzido do conjunto de dados.  
- Pode ser necessário excluí-las ou combinar com outras variáveis.  

---

### Próximos Passos

#### Plano de Ação para Otimização dos Modelos

**Investigar os Resultados da Regressão Linear**  
- Os resultados com **R² = 1** na Regressão Linear sugerem uma avaliação cuidadosa.  
- É necessário verificar se esse desempenho reflete um comportamento realista ou indica problemas nos dados, como redundâncias ou superajuste.  

**Ajustar Hiperparâmetros para Random Forest e XGBoost**  
- Embora esses modelos tenham demonstrado bom desempenho, ajustes nos hiperparâmetros podem melhorar ainda mais sua eficiência.  
- As configurações a serem exploradas incluem:  
  - **Número de estimadores (n_estimators):** [50, 100, 200].  
  - **Profundidade máxima (max_depth):** [3, 5, 10, None].  
  - **Taxa de aprendizado (learning_rate):** [0.01, 0.1, 0.2] (aplicável ao XGBoost).  

**Analisar Variáveis com Poucos Dados**  
- Algumas variáveis, como **Gini index** e **Income share held by lowest 20%**, possuem conjuntos de dados muito pequenos.  
- Ações possíveis incluem:  
  - Aplicar agregação ou imputação para aumentar a amostragem.  
  - Decidir se essas variáveis devem ser excluídas ou tratadas separadamente.  

**Selecionar os Melhores Modelos**  
- Com base nas métricas **R²**, **RMSE** e **MAE**, será possível identificar o modelo mais adequado para cada variável-alvo.  
- Essa etapa é essencial para assegurar o desempenho e a confiabilidade do modelo preditivo.  

**Gerar e Salvar Métricas Consolidadas**  
- Consolidar as métricas finais em um **DataFrame** e salvá-las para referência futura.  
- Isso garante que os resultados possam ser analisados de forma organizada e clara.  

**Visualizar os Resultados**  
- Criar gráficos comparativos para as métricas **R²**, **RMSE** e **MAE**, permitindo uma análise visual dos desempenhos e identificação de padrões ou discrepâncias entre os modelos.  

---

#### Plano de Ação Imediato

- Ajustar os hiperparâmetros para Random Forest e XGBoost conforme os valores sugeridos.  
- Consolidar os resultados em um DataFrame após os ajustes.  
- Avaliar o desempenho e registrar observações.  
- Decidir ações específicas para variáveis com poucos dados.  
- Documentar todas as etapas realizadas e atualizar o planejamento.  

Com essa abordagem estruturada, é possível otimizar o desempenho dos modelos e garantir que o processo seja bem documentado e transparente.  


In [25]:
# Parâmetros para ajuste
parametros_rf = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 10, None]}
parametros_xgb = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 10], 'learning_rate': [0.01, 0.1, 0.2]}

# Resultados ajustados
resultados_ajustados = []

# Ajustar modelos para cada variável-alvo
for target_column in variaveis_alvo:
    print(f"\nAjustando modelos para a variável-alvo: {target_column}")
    
    # Filtrar os dados para a variável-alvo
    dados_filtrados = dados[dados["Indicator Name"] == target_column]
    
    # Preparar X e y
    X = dados_filtrados.iloc[:, 4:].T  # Transpor para obter anos como linhas
    y = dados_filtrados.iloc[:, 4:].mean(axis=0)  # Média por ano
    
    # Ajustar índices para consistência
    X.columns = dados_filtrados["Country Name"].values
    y.index = X.index

    # Remover valores ausentes
    X = X.dropna(axis=1)
    y = y[~np.isnan(y)]
    
    # Garantir consistência entre X e y
    X = X.loc[y.index]
    
    # Dividir os dados em treino e teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Normalizar os dados
    X_treino = (X_treino - X_treino.mean()) / X_treino.std()
    X_teste = (X_teste - X_teste.mean()) / X_teste.std()
    
    # Ajustar Random Forest
    grid_rf = GridSearchCV(RandomForestRegressor(random_state=42), parametros_rf, cv=3, scoring='neg_mean_squared_error')
    grid_rf.fit(X_treino, y_treino)
    rf_melhor_modelo = grid_rf.best_estimator_
    rf_previsoes = rf_melhor_modelo.predict(X_teste)
    rf_r2 = r2_score(y_teste, rf_previsoes)
    rf_rmse = np.sqrt(mean_squared_error(y_teste, rf_previsoes))
    rf_mae = mean_absolute_error(y_teste, rf_previsoes)
    
    # Ajustar XGBoost
    grid_xgb = GridSearchCV(XGBRegressor(random_state=42, verbosity=0), parametros_xgb, cv=3, scoring='neg_mean_squared_error')
    grid_xgb.fit(X_treino, y_treino)
    xgb_melhor_modelo = grid_xgb.best_estimator_
    xgb_previsoes = xgb_melhor_modelo.predict(X_teste)
    xgb_r2 = r2_score(y_teste, xgb_previsoes)
    xgb_rmse = np.sqrt(mean_squared_error(y_teste, xgb_previsoes))
    xgb_mae = mean_absolute_error(y_teste, xgb_previsoes)
    
    # Salvar resultados
    resultados_ajustados.append({
        "Variável Alvo": target_column,
        "Modelo": "Random Forest",
        "R²": rf_r2,
        "RMSE": rf_rmse,
        "MAE": rf_mae
    })
    resultados_ajustados.append({
        "Variável Alvo": target_column,
        "Modelo": "XGBoost",
        "R²": xgb_r2,
        "RMSE": xgb_rmse,
        "MAE": xgb_mae
    })
    print(f"Random Forest - R²: {rf_r2:.3f}, RMSE: {rf_rmse:.3f}, MAE: {rf_mae:.3f}")
    print(f"XGBoost - R²: {xgb_r2:.3f}, RMSE: {xgb_rmse:.3f}, MAE: {xgb_mae:.3f}")

# Converter resultados em DataFrame
df_resultados_ajustados = pd.DataFrame(resultados_ajustados)

# Salvar resultados
df_resultados_ajustados.to_csv("resultados_ajustados.csv", index=False)
print("\nAjustes concluídos. Resultados salvos em 'resultados_ajustados.csv'.")


Ajustando modelos para a variável-alvo: CO2 emissions (metric tons per capita)
Random Forest - R²: -0.062, RMSE: 5.618, MAE: 2.636
XGBoost - R²: -0.060, RMSE: 5.613, MAE: 2.703

Ajustando modelos para a variável-alvo: Methane emissions (metric tons of CO2 equivalent per capita)
Random Forest - R²: 0.955, RMSE: 2.059, MAE: 1.390
XGBoost - R²: 0.950, RMSE: 2.184, MAE: 1.570

Ajustando modelos para a variável-alvo: Nitrous oxide emissions (metric tons of CO2 equivalent per capita)
Random Forest - R²: 0.963, RMSE: 1.965, MAE: 1.103
XGBoost - R²: 0.959, RMSE: 2.064, MAE: 1.223

Ajustando modelos para a variável-alvo: Energy use (kg of oil equivalent per capita)
Random Forest - R²: 0.746, RMSE: 470.102, MAE: 347.484
XGBoost - R²: 0.344, RMSE: 755.210, MAE: 572.439

Ajustando modelos para a variável-alvo: Fossil fuel energy consumption (% of total)
Random Forest - R²: 0.894, RMSE: 7.187, MAE: 4.680
XGBoost - R²: 0.883, RMSE: 7.554, MAE: 4.983

Ajustando modelos para a variável-alvo: Electric

Os resultados indicam que os ajustes nos hiperparâmetros trouxeram melhorias significativas para algumas variáveis-alvo, enquanto outras ainda apresentam desafios de desempenho, dependendo do modelo aplicado:

---

### Observações dos Modelos

**Modelos com Alto Desempenho:**  
- Variáveis como `Methane emissions` e `Adjusted savings: natural resources depletion` apresentaram bons resultados com **Random Forest** e **XGBoost**, sugerindo que esses modelos são robustos para padrões não lineares.  
- **XGBoost** mostrou desempenho competitivo para variáveis complexas, como `Patent applications`.  

**Modelos com Rendimento Inferior:**  
- Algumas variáveis, como `Energy use` e `Forest area`, apresentaram **RMSE** elevados ou **R²** negativos para **XGBoost**.  
  - Isso pode indicar necessidade de ajustes adicionais ou limitações do modelo para esses tipos de dados.  

**R² Negativo:**  
- Variáveis como `CO2 emissions` tiveram **R²** negativo para ambos os modelos, sugerindo discrepâncias significativas entre previsões e valores reais.  

---

### Próximos Passos

**Analisar Resultados:**  
- Explorar por que algumas variáveis tiveram **R²** negativos ou baixos.  
- Examinar a distribuição dos dados para identificar padrões, outliers ou inconsistências.  

**Refinamento de Hiperparâmetros:**  
- Para variáveis com desempenho médio, experimente ranges mais amplos de hiperparâmetros.  
- Adicionar mais valores de **learning_rate** e **max_depth** para **XGBoost**.  

**Modelos Alternativos:**  
- Testar outros algoritmos, como **Gradient Boosting (LightGBM)** ou **Redes Neurais**, para variáveis com desempenhos inconsistentes.  

**Documentação e Comparação:**  
- Consolidar os resultados em gráficos de **R²**, **RMSE** e **MAE** para uma análise visual clara.  
- Identificar os melhores modelos para cada variável-alvo com base nas métricas.  

**Iteração Final:**  
- Reavaliar o desempenho com o melhor conjunto de hiperparâmetros para cada modelo.  
- Decidir sobre os modelos finais para cada variável.  

**Preparação para o Deploy:**  
- Criar pipelines automatizados para facilitar o uso do modelo selecionado.  

In [30]:
# Caminho para o dataset ajustado
caminho_dados_ajustados = "../data/resultados_ajustados.csv"

# Carregar o dataset ajustado
dados = pd.read_csv(caminho_dados_ajustados)

print(dados.columns)

Index(['Variável Alvo', 'Modelo', 'R²', 'RMSE', 'MAE'], dtype='object')


In [32]:
# Parâmetros para ajuste de hiperparâmetros
parametros_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}

parametros_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Lista de variáveis-alvo
variaveis_alvo = dados["Variável Alvo"].unique()

# DataFrame para armazenar resultados
resultados_ajustes = []

# Ajustar hiperparâmetros para cada variável-alvo
for target_column in variaveis_alvo:
    print(f"\nAjustando modelos para a variável-alvo: {target_column}")
    
    # Filtrar dados para a variável-alvo
    dados_target = dados[dados["Variável Alvo"] == target_column]
    
    # Preparar os dados
    X = dados_target.drop(columns=["Variável Alvo", "Modelo", "R²", "RMSE", "MAE"])  # Features
    y = dados_target["RMSE"]  # Variável alvo (RMSE no caso)

    # Certificar-se de que X e y possuem os mesmos índices
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    # Verificar se há amostras suficientes para dividir os dados
    if len(y) < 3:
        print(f"A variável-alvo '{target_column}' tem apenas {len(y)} amostras. Pulando ajuste.")
        continue
    
    # Dividir os dados em treino e teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Ajustar Random Forest
    try:
        grid_rf = GridSearchCV(RandomForestRegressor(random_state=42), parametros_rf, cv=min(3, len(y_treino)), scoring='neg_mean_squared_error')
        grid_rf.fit(X_treino, y_treino)
        melhor_rf = grid_rf.best_estimator_
        previsoes_rf = melhor_rf.predict(X_teste)

        r2_rf = r2_score(y_teste, previsoes_rf)
        rmse_rf = np.sqrt(mean_squared_error(y_teste, previsoes_rf))
        mae_rf = mean_absolute_error(y_teste, previsoes_rf)

        resultados_ajustes.append({
            "Variável Alvo": target_column,
            "Modelo": "Random Forest",
            "Melhores Hiperparâmetros": grid_rf.best_params_,
            "R²": r2_rf,
            "RMSE": rmse_rf,
            "MAE": mae_rf
        })
        print(f"Random Forest: R²={r2_rf:.3f}, RMSE={rmse_rf:.3f}, MAE={mae_rf:.3f}")
    except Exception as e:
        print(f"Erro ao ajustar Random Forest para '{target_column}': {e}")
    
    # Ajustar XGBoost
    try:
        grid_xgb = GridSearchCV(XGBRegressor(random_state=42, verbosity=0), parametros_xgb, cv=min(3, len(y_treino)), scoring='neg_mean_squared_error')
        grid_xgb.fit(X_treino, y_treino)
        melhor_xgb = grid_xgb.best_estimator_
        previsoes_xgb = melhor_xgb.predict(X_teste)

        r2_xgb = r2_score(y_teste, previsoes_xgb)
        rmse_xgb = np.sqrt(mean_squared_error(y_teste, previsoes_xgb))
        mae_xgb = mean_absolute_error(y_teste, previsoes_xgb)

        resultados_ajustes.append({
            "Variável Alvo": target_column,
            "Modelo": "XGBoost",
            "Melhores Hiperparâmetros": grid_xgb.best_params_,
            "R²": r2_xgb,
            "RMSE": rmse_xgb,
            "MAE": mae_xgb
        })
        print(f"XGBoost: R²={r2_xgb:.3f}, RMSE={rmse_xgb:.3f}, MAE={mae_xgb:.3f}")
    except Exception as e:
        print(f"Erro ao ajustar XGBoost para '{target_column}': {e}")

# Consolidar e salvar os resultados
df_resultados_ajustes = pd.DataFrame(resultados_ajustes)
df_resultados_ajustes.to_csv("../data/resultados_ajustes_finais.csv", index=False)
print("\nAjustes concluídos. Resultados salvos em '../data/resultados_ajustes_finais.csv'.")


Ajustando modelos para a variável-alvo: CO2 emissions (metric tons per capita)
A variável-alvo 'CO2 emissions (metric tons per capita)' tem apenas 2 amostras. Pulando ajuste.

Ajustando modelos para a variável-alvo: Methane emissions (metric tons of CO2 equivalent per capita)
A variável-alvo 'Methane emissions (metric tons of CO2 equivalent per capita)' tem apenas 2 amostras. Pulando ajuste.

Ajustando modelos para a variável-alvo: Nitrous oxide emissions (metric tons of CO2 equivalent per capita)
A variável-alvo 'Nitrous oxide emissions (metric tons of CO2 equivalent per capita)' tem apenas 2 amostras. Pulando ajuste.

Ajustando modelos para a variável-alvo: Energy use (kg of oil equivalent per capita)
A variável-alvo 'Energy use (kg of oil equivalent per capita)' tem apenas 2 amostras. Pulando ajuste.

Ajustando modelos para a variável-alvo: Fossil fuel energy consumption (% of total)
A variável-alvo 'Fossil fuel energy consumption (% of total)' tem apenas 2 amostras. Pulando ajuste